In [2]:
성북구 = [1129010100,1129010200,1129010300,1129010400,1129010500,1129010600,1129010700,1129010800,1129010900,1129011000,1129011100,1129011200,1129011300,1129011400,1129011500,1129011600,1129011700,1129011800,1129011900,1129012000,1129012100,1129012200,1129012300,1129012400,1129012500,1129012600,1129012700,1129012800,1129012900,1129013000,1129013100,1129013200,1129013300,1129013400,1129013500,1129013600,1129013700,1129013800,1129013900]
seongbuk = ['성북동','성북동1가','돈암동','동소문동1가','동소문동2가','동소문동3가','동소문동4가','동소문동5가','동소문동6가','동소문동7가','삼선동1가','삼선동2가','삼선동3가','삼선동4가','삼선동5가','동선동1가','동선동2가','동선동3가','동선동4가','동선동5가','안암동1가','안암동2가','안암동3가','안암동4가','안암동5가','보문동4가','보문동5가','보문동6가','보문동7가','보문동1가','보문동2가','보문동3가','정릉동','길음동','종암동','하월곡동','상월곡동','장위동','석관동']

In [3]:
import requests #필요한 모듈들을 import 합니다.
from bs4 import BeautifulSoup
import pandas as pd

names = []
areas = []
prices = []
area_per_price = []
for area_code in 성북구:
    
    url = 'https://hogangnono.com/region/{}'.format(area_code) #url을 area_code를 이용하여 반복문으로 한번에 모두 가져옵니다.
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    name = []
    for pages in soup.findAll('li'):    # 아파트 이름을 가져옵니다
        for link in pages('h5',{'class':'name'}):
            data = link.text        
            name.append(data)
    area = []    
    for pages in soup.findAll('li'):    # 아프트 면적을 가져옵니다
        for link in pages('span',{'class':'area'}):
            data = link.text        
            area.append(data)
    price = []
    for pages in soup.findAll('li'):    #아파트 가겨을 가져옵니다
        for link in pages('span',{'class':'price'}):
            data = link.text        
            price.append(data)
            
    index_dash = [i for i, n in enumerate(price) if n == '-'] # 아파트 가격이 '-' 로 나와있는 아파트들의 index를 찾아냅니다
    for i in index_dash[::-1]: # '-'로 나와있는 아파트를 리스트에서 지웁니다
        del(name[i])
        del(area[i])
        del(price[i])
    
    new_name = [] # 아파트 이름 앞에 동 이름을 제거합니다
    for k in name:
        k_idx = k.find(' ')
        k = k[k_idx+1:]
        new_name.append(k)
        
    
    int_area = []
    for l in area: # 아파트 면적을 integer로 바꿔줍니다
        per = l[:-1]
        int_area.append(int(per))
    
    new_price = []
    for j in price: # 아파트 가격을 integer 로 바꿔줍니다

        if '억'in j:
            if len(j)>4:
                y_idx = j.find('억')
                pri = j[0:y_idx]
                pri1 = j[y_idx+1:]
                pri2 = pri1.replace(',','')
                j = ((int(pri)*10000)+(int(pri2)))*10000
                new_price.append(j)
            else:
                j = j.replace('억','')
                j = int(j)*100000000
                new_price.append(j)

        else:
            j = j.replace(',','')
            j = int(j)*10000
            new_price.append(j)
            
    per_price = [] # 평당 가격을 계산합니다
    for a in range(len(new_price)):
        perp = new_price[a]/int_area[a]
        per_price.append(round(perp))
        
    
    
    names.append(new_name) # lists in lists 로 저장합니다
    areas.append(int_area)
    prices.append(new_price)
    area_per_price.append(per_price)
    
amean = []

for i in range(len(seongbuk)): # 동별로 평당가격의 mean을 구해줍니다
    if len(area_per_price[i])>0:
        mean = sum(area_per_price[i])/len(area_per_price[i])
        amean.append(round(mean))
        
    else:
        amean.append(0)

kk = {'dong':성북구,'per price':amean}
df = pd.DataFrame(kk,index = seongbuk)
df

,dong,per price
성북동,1129010100,17234848
성북동1가,1129010200,26000000
돈암동,1129010300,18783772
동소문동1가,1129010400,0
동소문동2가,1129010500,0
동소문동3가,1129010600,0
동소문동4가,1129010700,14749415
동소문동5가,1129010800,27324885
동소문동6가,1129010900,20972222
동소문동7가,1129011000,24125000


In [4]:
df.to_csv('seongbukgu.csv')